In [2]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import os
import sys
import plotly.graph_objects as go 
dir_path = os.path.abspath('')
sys.path.append(dir_path + '/../')
from labbiofisica import Interpolazione, final_val
from scipy.optimize import curve_fit

In [14]:
LAMBDA_ECC = np.array([500,510,520,490,530,526,480,470,515,460])
filename = 'data/ecc_rodamina_05.csv'
data = pd.read_csv(filename, sep=',', header=1,nrows=151)
data = data.drop(columns=['Unnamed: 20'])
data.columns = [f'λ {LAMBDA_ECC[i]}' if j % 2 == 0 else f'I {LAMBDA_ECC[i]}' for i in range(len(LAMBDA_ECC)) for j in range(2)]
# Drop unnamed
data.tail()

,λ 500,I 500,λ 510,I 510,λ 520,I 520,λ 490,I 490,λ 530,I 530,λ 526,I 526,λ 480,I 480,λ 470,I 470,λ 515,I 515,λ 460,I 460
146,650.859985,12.056331,NaN,NaN,NaN,NaN,641.010010,15.735465,NaN,NaN,NaN,NaN,631.010010,17.525221,621.020019,16.186842,NaN,NaN,611.020019,16.775307
147,NaN,NaN,NaN,NaN,NaN,NaN,642.020019,15.055171,NaN,NaN,NaN,NaN,632.020019,16.700592,622.049988,15.503691,NaN,NaN,612.049988,16.016869
148,NaN,NaN,NaN,NaN,NaN,NaN,643.039978,14.409358,NaN,NaN,NaN,NaN,633.039978,15.951087,622.940002,14.866682,NaN,NaN,612.940002,15.428048
149,NaN,NaN,NaN,NaN,NaN,NaN,644.049988,13.756996,NaN,NaN,NaN,NaN,634.049988,15.206162,623.969971,14.247660,NaN,NaN,613.969971,14.786211
150,NaN,NaN,NaN,NaN,NaN,NaN,645.070007,13.192753,NaN,NaN,NaN,NaN,635.070007,14.392628,625.000000,13.497140,NaN,NaN,615.000000,14.177029


In [ ]:
for i,λ in enumerate(LAMBDA_ECC):
    data[f'Wavelengh{i}'] = λ
    data[f'λ_{i}'] = data[f'λ_{i}'].astype(float)